In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rts.utils import dataframe_from_hdf5
from rts.features.pose import load_all_poses, draw_pose
import umap
import cv2

from sklearn.cluster import DBSCAN, KMeans

archive_path = "/mnt/g/ioc/data/"
poses_folder = "data/test_poses/"
sequences_folder = "/mnt/g/ioc/sequences/"

In [ ]:
poses = load_all_poses(poses_folder)
data = dataframe_from_hdf5(archive_path, "metadata")

pose_df = pd.merge(pd.DataFrame(poses), data[["seq_id", "sport"]], left_on="video_name", right_on="seq_id")

# Similar poses

In [ ]:
# Find nearest neighbors

from sklearn.neighbors import NearestNeighbors

def get_nearest_neighbors(input_pose, n_neighbors=5, dist_threshold = 0.05):
    other_poses = pose_df[pose_df["video_name"] != input_pose["video_name"]].reset_index(drop=True)
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(other_poses["angle_vec"].tolist())
    distances, indices = nbrs.kneighbors([input_pose["angle_vec"]])
    results = other_poses.iloc[indices[0]].reset_index(drop=True)
    results["distance"] = distances[0]
    results = results[results["distance"] < dist_threshold]
    return results

In [ ]:
n_neighbors = 5
for i in range(5):
    input_pose = pose_df.sample(1).iloc[0]
    results = get_nearest_neighbors(input_pose, n_neighbors=n_neighbors)

    fig, axs = plt.subplots(nrows=1, ncols=n_neighbors + 1, figsize=((n_neighbors + 1) * 3, 3))
    axs = axs.flatten()
    draw_pose(input_pose, video_folder=sequences_folder, ax = axs[0], cut = True)
    axs[0].set_title("Input pose")
    for i, pose in results.iterrows():
        draw_pose(pose, video_folder=sequences_folder, ax = axs[i+1], cut = True)
    plt.show()

# Matching Analytics

In [ ]:
def matching_analytics(input_pose):
    results = get_nearest_neighbors(input_pose, n_neighbors=100, dist_threshold=0.05)
    results["sport"].value_counts()

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), gridspec_kw={'width_ratios': [1, 3]})
    axs = axs.flatten()
    axs[0].boxplot(results["distance"])
    axs[0].set_title("Distances")

    sports_counts = results["sport"].value_counts()
    axs[1].bar(sports_counts.index, sports_counts.values)
    axs[1].set_title("Sports (n=%d)" % len(results))
    axs[1].set_xticklabels(sports_counts.index, rotation=45)

    plt.suptitle("Nearest neighbors analytics for %s (%s)" % (input_pose["video_name"], input_pose["sport"]))
    plt.show()

In [ ]:
input_pose = pose_df.sample(1).iloc[0]

matching_analytics(input_pose)